In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Otros objetivos
# -----------------------------------------------------------------------
import math

# Gráficos
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

import os
import sys 

import warnings
warnings.filterwarnings("ignore")



# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor
from scipy import stats

import pickle
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler, RobustScaler


pd.options.display.float_format = "{:,.2f}".format 

# import plotly_express as px


# Métodos estadísticos
# -----------------------------------------------------------------------
from scipy.stats import zscore # para calcular el z-score
from sklearn.neighbors import LocalOutlierFactor # para detectar outliers usando el método LOF
from sklearn.ensemble import IsolationForest # para detectar outliers usando el metodo IF
from sklearn.neighbors import NearestNeighbors # para calcular la epsilon

# Para generar combinaciones de listas
# -----------------------------------------------------------------------
from itertools import product, combinations


from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Para la codificación de las variables numéricas
# -----------------------------------------------------------------------
from sklearn.preprocessing import OneHotEncoder #, TargetEncoder # para poder aplicar los métodos de OneHot, Ordinal,  Label y Target Encoder 

sys.path.append(os.path.abspath("../../src"))   
import Soporte_encoding as f
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder     #solo me deja hacerlo si uso el entorno default en vez del proyecto
pd.set_option("display.max_columns", None)


In [2]:
df_sinull=pd.read_pickle("../../datos/datos1/datos_sin_nulos.pkl").reset_index(drop=True)


In [3]:
df_sinull.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement
0,51.00,0.00,Travel_Rarely,Sales,entre 5 y 8,College,Life Sciences,Female,Intern,Healthcare Representative,Married,"1,482.11",1,11.00,Bad,1,6.00,1.00,0.00,High,Very High,Good,High
1,31.00,1.00,Travel_Frequently,Research & Development,entre 9 y 12,Below College,Life Sciences,Female,Intern,Research Scientist,Single,473.36,0,23.00,Good,6,3.00,5.00,1.00,High,Medium,Best,Medium
2,32.00,0.00,Travel_Frequently,Research & Development,entre 13 y 18,Master,Other,Male,Manager,Sales Executive,Married,"2,184.06",1,15.00,Best,5,2.00,5.00,0.00,Medium,Medium,Bad,High
3,38.00,0.00,Non-Travel,Research & Development,entre 1 y 4,Doctor,Life Sciences,Male,Senior,Human Resources,Married,940.27,3,11.00,Best,13,5.00,8.00,7.00,Very High,Very High,Better,Medium
4,32.00,0.00,Travel_Rarely,Research & Development,entre 9 y 12,Below College,Medical,Male,Intern,Sales Executive,Single,264.65,4,12.00,Better,9,2.00,6.00,0.00,Very High,Low,Better,High


ah


In [4]:
df_sinull_copy=df_sinull.copy()

In [5]:
df_categoricas= df_sinull_copy.select_dtypes(include=["category","object"])

In [6]:
df_categoricas

,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,NumCompaniesWorked,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement
0,Travel_Rarely,Sales,entre 5 y 8,College,Life Sciences,Female,Intern,Healthcare Representative,Married,1,Bad,1,6.00,High,Very High,Good,High
1,Travel_Frequently,Research & Development,entre 9 y 12,Below College,Life Sciences,Female,Intern,Research Scientist,Single,0,Good,6,3.00,High,Medium,Best,Medium
2,Travel_Frequently,Research & Development,entre 13 y 18,Master,Other,Male,Manager,Sales Executive,Married,1,Best,5,2.00,Medium,Medium,Bad,High
3,Non-Travel,Research & Development,entre 1 y 4,Doctor,Life Sciences,Male,Senior,Human Resources,Married,3,Best,13,5.00,Very High,Very High,Better,Medium
4,Travel_Rarely,Research & Development,entre 9 y 12,Below College,Medical,Male,Intern,Sales Executive,Single,4,Better,9,2.00,Very High,Low,Better,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,Travel_Rarely,Research & Development,entre 5 y 8,Master,Medical,Female,Intern,Research Scientist,Single,3,Good,10,5.00,Very High,Low,Better,High
4406,Travel_Rarely,Research & Development,entre 1 y 4,Master,Medical,Male,Intern,Laboratory Technician,Divorced,2,Bad,10,2.00,Very High,Very High,Better,Medium
4407,Travel_Rarely,Research & Development,entre 24 y 29,College,Life Sciences,Male,Junior,Sales Executive,Married,0,Bad,5,4.00,Low,High,Better,High
4408,Travel_Rarely,Sales,entre 13 y 18,College,Medical,Male,Intern,Laboratory Technician,Divorced,0,Good,10,2.00,Very High,Low,Better,Medium


In [7]:
df_categoricas

,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,NumCompaniesWorked,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement
0,Travel_Rarely,Sales,entre 5 y 8,College,Life Sciences,Female,Intern,Healthcare Representative,Married,1,Bad,1,6.00,High,Very High,Good,High
1,Travel_Frequently,Research & Development,entre 9 y 12,Below College,Life Sciences,Female,Intern,Research Scientist,Single,0,Good,6,3.00,High,Medium,Best,Medium
2,Travel_Frequently,Research & Development,entre 13 y 18,Master,Other,Male,Manager,Sales Executive,Married,1,Best,5,2.00,Medium,Medium,Bad,High
3,Non-Travel,Research & Development,entre 1 y 4,Doctor,Life Sciences,Male,Senior,Human Resources,Married,3,Best,13,5.00,Very High,Very High,Better,Medium
4,Travel_Rarely,Research & Development,entre 9 y 12,Below College,Medical,Male,Intern,Sales Executive,Single,4,Better,9,2.00,Very High,Low,Better,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,Travel_Rarely,Research & Development,entre 5 y 8,Master,Medical,Female,Intern,Research Scientist,Single,3,Good,10,5.00,Very High,Low,Better,High
4406,Travel_Rarely,Research & Development,entre 1 y 4,Master,Medical,Male,Intern,Laboratory Technician,Divorced,2,Bad,10,2.00,Very High,Very High,Better,Medium
4407,Travel_Rarely,Research & Development,entre 24 y 29,College,Life Sciences,Male,Junior,Sales Executive,Married,0,Bad,5,4.00,Low,High,Better,High
4408,Travel_Rarely,Sales,entre 13 y 18,College,Medical,Male,Intern,Laboratory Technician,Divorced,0,Good,10,2.00,Very High,Low,Better,Medium


Tenemos 17 columnas de categóricas

In [8]:
df_categoricas.nunique()

BusinessTravel              3
Department                  3
DistanceFromHome            6
Education                   5
EducationField              6
Gender                      2
JobLevel                    5
JobRole                     9
MaritalStatus               3
NumCompaniesWorked         10
StockOptionLevel            4
TotalWorkingYears          40
TrainingTimesLastYear       7
EnvironmentSatisfaction     4
JobSatisfaction             4
WorkLifeBalance             4
JobInvolvement              4
dtype: int64

In [9]:
df_categoricas.head()

,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,NumCompaniesWorked,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement
0,Travel_Rarely,Sales,entre 5 y 8,College,Life Sciences,Female,Intern,Healthcare Representative,Married,1,Bad,1,6.00,High,Very High,Good,High
1,Travel_Frequently,Research & Development,entre 9 y 12,Below College,Life Sciences,Female,Intern,Research Scientist,Single,0,Good,6,3.00,High,Medium,Best,Medium
2,Travel_Frequently,Research & Development,entre 13 y 18,Master,Other,Male,Manager,Sales Executive,Married,1,Best,5,2.00,Medium,Medium,Bad,High
3,Non-Travel,Research & Development,entre 1 y 4,Doctor,Life Sciences,Male,Senior,Human Resources,Married,3,Best,13,5.00,Very High,Very High,Better,Medium
4,Travel_Rarely,Research & Development,entre 9 y 12,Below College,Medical,Male,Intern,Sales Executive,Single,4,Better,9,2.00,Very High,Low,Better,High


Ahora debemos comprobar la normalidad y homocedasticidad para saber si son parametricas o no parametricas (Con el fin de más adelante poder evaluar si hay diferencias significativas)

Tuve que convertir estas variables a str porque sino no las detectaba como categóricas

In [10]:
# df_sinout_copy["exterior"]=df_sinout_copy["exterior"].astype(str)
# df_sinout_copy["rooms"]=df_sinout_copy["rooms"].astype(str)
# df_sinout_copy["bathrooms"]=df_sinout_copy["bathrooms"].astype(str)

In [11]:
f.detectar_orden_cat(df_sinull_copy,df_categoricas.columns,"Attrition")

Estamos evaluando el orden de la variable BUSINESSTRAVEL


Attrition,0.00,1.00
BusinessTravel,,
Non-Travel,414,36
Travel_Frequently,624,207
Travel_Rarely,2661,468


La variable BusinessTravel SI tiene orden
Estamos evaluando el orden de la variable DEPARTMENT


Attrition,0.00,1.00
Department,,
Human Resources,132,57
Research & Development,2430,453
Sales,1137,201


La variable Department SI tiene orden
Estamos evaluando el orden de la variable DISTANCEFROMHOME


...

La variable JobSatisfaction SI tiene orden
Estamos evaluando el orden de la variable WORKLIFEBALANCE


Attrition,0.00,1.00
WorkLifeBalance,,
Bad,164,75
Best,373,81
Better,2311,384
Good,851,171


La variable WorkLifeBalance SI tiene orden
Estamos evaluando el orden de la variable JOBINVOLVEMENT


Attrition,0.00,1.00
JobInvolvement,,
Low,195,54
Medium,945,180
High,2205,399
Very High,354,78


La variable JobInvolvement SI tiene orden


#### CONCLUSIONES SOBRE EL ORDEN:  

**NO ORDENADAS:**
- Gender  
- DistanceFromHome  
- Education   
- JobLevel  
- StockOptionLevel  

  
 *- get- dummies (pandas)*  
 *- One-Hot Encoding (SKlearn) (es más efectivo)*  
 *Ambas te crean tantas columnas como categorias hay en la columna y hace que en esa solo valga 1 las que coincidan y 0 el resto*


**ORDENADAS:**  
- BusinessTravel 
- Department  
- JobRole  
- MaritalStatus  
- NumCompaniesWorked  
- TrainingTimesLastYear  
- EnvironmentSatisfaction  
- JobSatisfaction  
- WorkLifeBalance  
- JobInvolvement  
- EducationField  
- TotalWorkingYears  
  

*lo de entre parentesis hace referencia a la cantidad de categorías que son diferentes en esa columna*  

*- Ordinal Encoding  (Los codifica en función de los pesos que le digamos)*
*- Label Encoding (Los codifica en orden alfabetico)*  
*- Target Encoding (Los codifica en funcion de las medias (osea en función de la relación con la variable dependiente))*  
*- Frequency Encoding (Los codifica en funcion de la frecuencia(en función del value_counts))*  



**Ahora pasamos al encodin de las que no tienen orden:**

In [12]:
df_sinull_copy.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'Gender', 'JobLevel', 'JobRole',
       'MaritalStatus', 'MonthlyIncome', 'NumCompaniesWorked',
       'PercentSalaryHike', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion',
       'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance',
       'JobInvolvement'],
      dtype='object')

In [13]:
diccionario_encoding = {"onehot": ["Gender", "DistanceFromHome", "Education", "JobLevel", "StockOptionLevel"], 
                        "dummies": [], # no metemos ninguna
                        'ordinal' : { }, #no metemos ninguna
                        "label": [] , # no metemos ninguna columna porque no queremos en ningún caso que se asignen las categorías de forma aleatoria
                        "frequency": [], # no metemos ninguna columna porque no coincide el orden del value counts con las categorias y la variable respuesta
                        "target": ["BusinessTravel", "Department", "JobRole", "MaritalStatus","NumCompaniesWorked", "TrainingTimesLastYear", "EnvironmentSatisfaction", "JobSatisfaction", "WorkLifeBalance", "JobInvolvement", "EducationField", "TotalWorkingYears"]  
                        }

In [14]:
cols_onehot = ["Gender", "DistanceFromHome", "Education", "JobLevel", "StockOptionLevel"]
one_hot_encoder = OneHotEncoder(categories='auto', 
                        drop=None, 
                        sparse_output=True, 
                        dtype='float', 
                        handle_unknown='error')

encoder_trans = one_hot_encoder.fit_transform(df_sinull_copy[cols_onehot])
encoder_array = encoder_trans.toarray()

# usamos el método get_feature_names_out() para extraer el nombre de las columnas
nombre_columnas = one_hot_encoder.get_feature_names_out()

encoder_df = pd.DataFrame(encoder_array, columns = nombre_columnas)
df_sinull_copy = pd.concat([df_sinull_copy, encoder_df], axis = 1)
df_sinull_copy.drop(columns=cols_onehot, inplace=True)


In [15]:

with open('../../transformers/one_hot_encoder.pkl', 'wb') as o:
    pickle.dump(one_hot_encoder, o)
    


In [16]:
one_hot_encoder.categories_

[array(['Female', 'Male'], dtype=object),
 array(['entre 1 y 4', 'entre 13 y 18', 'entre 19 y 23', 'entre 24 y 29',
        'entre 5 y 8', 'entre 9 y 12'], dtype=object),
 array(['Bachelor', 'Below College', 'College', 'Doctor', 'Master'],
       dtype=object),
 array(['Head', 'Intern', 'Junior', 'Manager', 'Senior'], dtype=object),
 array(['Bad', 'Best', 'Better', 'Good'], dtype=object)]

In [17]:
df_sinull_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 40 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   Age                             4410 non-null   float64 
 1   Attrition                       4410 non-null   float64 
 2   BusinessTravel                  4410 non-null   object  
 3   Department                      4410 non-null   object  
 4   EducationField                  4410 non-null   object  
 5   JobRole                         4410 non-null   object  
 6   MaritalStatus                   4410 non-null   object  
 7   MonthlyIncome                   4410 non-null   float64 
 8   NumCompaniesWorked              4410 non-null   object  
 9   PercentSalaryHike               4410 non-null   float64 
 10  TotalWorkingYears               4410 non-null   object  
 11  TrainingTimesLastYear           4410 non-null   category
 12  YearsAtCompany      

In [18]:
target_encoder= TargetEncoder(cols=["BusinessTravel", "Department", "JobRole", "MaritalStatus","NumCompaniesWorked", "TrainingTimesLastYear", "EnvironmentSatisfaction", "JobSatisfaction", "WorkLifeBalance", "JobInvolvement", "EducationField", "TotalWorkingYears"])
encoded= target_encoder.fit_transform(df_sinull_copy , df_sinull_copy[["Attrition"]])



In [19]:
with open('../../transformers/target_encoder.pkl', 'wb') as t:
    pickle.dump(target_encoder, t)

In [20]:
encoded.head(2)

,Age,Attrition,BusinessTravel,Department,EducationField,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,Gender_Female,Gender_Male,DistanceFromHome_entre 1 y 4,DistanceFromHome_entre 13 y 18,DistanceFromHome_entre 19 y 23,DistanceFromHome_entre 24 y 29,DistanceFromHome_entre 5 y 8,DistanceFromHome_entre 9 y 12,Education_Bachelor,Education_Below College,Education_College,Education_Doctor,Education_Master,JobLevel_Head,JobLevel_Intern,JobLevel_Junior,JobLevel_Manager,JobLevel_Senior,StockOptionLevel_Bad,StockOptionLevel_Best,StockOptionLevel_Better,StockOptionLevel_Good
0,51.00,0.00,0.15,0.15,0.17,0.15,0.12,"1,482.11",0.19,11.00,0.49,0.06,1.00,0.00,0.14,0.11,0.17,0.15,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
1,31.00,1.00,0.25,0.16,0.17,0.18,0.26,473.36,0.12,23.00,0.18,0.18,5.00,1.00,0.14,0.16,0.18,0.16,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00


In [21]:
encoded.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'EducationField',
       'JobRole', 'MaritalStatus', 'MonthlyIncome', 'NumCompaniesWorked',
       'PercentSalaryHike', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'EnvironmentSatisfaction',
       'JobSatisfaction', 'WorkLifeBalance', 'JobInvolvement', 'Gender_Female',
       'Gender_Male', 'DistanceFromHome_entre 1 y 4',
       'DistanceFromHome_entre 13 y 18', 'DistanceFromHome_entre 19 y 23',
       'DistanceFromHome_entre 24 y 29', 'DistanceFromHome_entre 5 y 8',
       'DistanceFromHome_entre 9 y 12', 'Education_Bachelor',
       'Education_Below College', 'Education_College', 'Education_Doctor',
       'Education_Master', 'JobLevel_Head', 'JobLevel_Intern',
       'JobLevel_Junior', 'JobLevel_Manager', 'JobLevel_Senior',
       'StockOptionLevel_Bad', 'StockOptionLevel_Best',
       'StockOptionLevel_Better', 'StockOptionLevel_Good'],
      dtype='object')

TENGO QUE REVISAR PORQUE ME HA GENERADO NAN EN MONTCREATED

In [22]:
encoded.head()

,Age,Attrition,BusinessTravel,Department,EducationField,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,Gender_Female,Gender_Male,DistanceFromHome_entre 1 y 4,DistanceFromHome_entre 13 y 18,DistanceFromHome_entre 19 y 23,DistanceFromHome_entre 24 y 29,DistanceFromHome_entre 5 y 8,DistanceFromHome_entre 9 y 12,Education_Bachelor,Education_Below College,Education_College,Education_Doctor,Education_Master,JobLevel_Head,JobLevel_Intern,JobLevel_Junior,JobLevel_Manager,JobLevel_Senior,StockOptionLevel_Bad,StockOptionLevel_Best,StockOptionLevel_Better,StockOptionLevel_Good
0,51.00,0.00,0.15,0.15,0.17,0.15,0.12,"1,482.11",0.19,11.00,0.49,0.06,1.00,0.00,0.14,0.11,0.17,0.15,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
1,31.00,1.00,0.25,0.16,0.17,0.18,0.26,473.36,0.12,23.00,0.18,0.18,5.00,1.00,0.14,0.16,0.18,0.16,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
2,32.00,0.00,0.25,0.16,0.12,0.17,0.12,"2,184.06",0.19,15.00,0.18,0.17,5.00,0.00,0.15,0.16,0.31,0.15,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00
3,38.00,0.00,0.08,0.16,0.17,0.13,0.12,940.27,0.10,11.00,0.09,0.14,8.00,7.00,0.13,0.11,0.14,0.16,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00
4,32.00,0.00,0.15,0.16,0.16,0.17,0.26,264.65,0.12,12.00,0.10,0.17,6.00,0.00,0.13,0.23,0.14,0.15,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00


In [23]:
encoded.to_pickle("../../datos/datos1/datos_sinnull_encoded.pkl")